<a href="https://colab.research.google.com/github/Aishu040/Aishu040/blob/main/model_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install --upgrade pip setuptools wheel

In [2]:
pip install --upgrade datasets seqeval


  Using cached datasets-3.3.2-py3-none-any.whl.metadata (19 kB)
  Using cached seqeval-1.2.2.tar.gz (43 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16217 sha256=72bad6103dfee1f551b12e83ff57f1d19980e87ef0b10fc387a57ca49826867f
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


In [3]:
pip install --upgrade datasets seqeval

In [4]:
pip install --no-cache-dir scikit-learn

In [5]:
import sklearn
print(sklearn.__version__)

1.6.1


In [6]:
import json
import torch
import numpy as np
from transformers import BertTokenizerFast, BertForTokenClassification, Trainer, TrainingArguments
from datasets import Dataset
from seqeval.metrics import classification_report
from sklearn.model_selection import train_test_split

In [7]:
with open("annotations.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [8]:
print(data)

[['I am a farmer from Tamil Nadu. My land was affected by floods, and my patta documents are missing. I have applied for a duplicate patta, but my application has been pending for 11 months. Without this document, I am unable to claim government compensation. I had filed complaints twice, but no action was taken.\r', {'entities': [[67, 135, 'ISSUE'], [165, 187, 'DELAY'], [216, 256, 'SEVERITY'], [263, 285, 'COMPLIANT RISED']]}], ['I am a farmer from Tamil Nadu. My land was affected by floods, and my patta documents are missing. I have applied for a duplicate patta, but my application has been pending for 12 months. Without this document, I am unable to claim government compensation. I had filed complaints twice, but no action was taken.\r', {'entities': [[67, 135, 'ISSUE'], [165, 187, 'DELAY'], [216, 256, 'SEVERITY'], [269, 285, 'COMPLIANT RISED']]}], ['I applied for a caste certificate for my child’s school admission 11 months ago. The delay in processing is causing problems in securin

In [10]:
for idx, (text, annotations) in enumerate(data):
    for entity in annotations["entities"]:
        start, end, label = entity
        if text[start:end] != text[start:end]:  # Check for mismatch
            print(f"Error: Entity boundary mismatch at index {idx}")

In [11]:
from transformers import BertTokenizer, BertForTokenClassification

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Add "patta" to the vocabulary
new_tokens = ["patta"]
tokenizer.add_tokens(new_tokens)

# Load the BERT model for NER
model = BertForTokenClassification.from_pretrained("bert-base-uncased", num_labels=5)  # Adjust num_labels as needed

# Resize the token embeddings to include the new token
model.resize_token_embeddings(len(tokenizer))

for text, annotations in data[:]:  # Check first two samples
    tokens = tokenizer.tokenize(text)
    print(tokens)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


['i', 'am', 'a', 'farmer', 'from', 'tamil', 'nadu', '.', 'my', 'land', 'was', 'affected', 'by', 'floods', ',', 'and', 'my', 'patta', 'documents', 'are', 'missing', '.', 'i', 'have', 'applied', 'for', 'a', 'duplicate', 'patta', ',', 'but', 'my', 'application', 'has', 'been', 'pending', 'for', '11', 'months', '.', 'without', 'this', 'document', ',', 'i', 'am', 'unable', 'to', 'claim', 'government', 'compensation', '.', 'i', 'had', 'filed', 'complaints', 'twice', ',', 'but', 'no', 'action', 'was', 'taken', '.']
['i', 'am', 'a', 'farmer', 'from', 'tamil', 'nadu', '.', 'my', 'land', 'was', 'affected', 'by', 'floods', ',', 'and', 'my', 'patta', 'documents', 'are', 'missing', '.', 'i', 'have', 'applied', 'for', 'a', 'duplicate', 'patta', ',', 'but', 'my', 'application', 'has', 'been', 'pending', 'for', '12', 'months', '.', 'without', 'this', 'document', ',', 'i', 'am', 'unable', 'to', 'claim', 'government', 'compensation', '.', 'i', 'had', 'filed', 'complaints', 'twice', ',', 'but', 'no', 'ac

In [12]:
print("Data Type:", type(data))
print("First Batch Type:", type(data[0]))
print("First Batch Content:", data[0])

Data Type: <class 'list'>
First Batch Type: <class 'list'>
First Batch Content: ['I am a farmer from Tamil Nadu. My land was affected by floods, and my patta documents are missing. I have applied for a duplicate patta, but my application has been pending for 11 months. Without this document, I am unable to claim government compensation. I had filed complaints twice, but no action was taken.\r', {'entities': [[67, 135, 'ISSUE'], [165, 187, 'DELAY'], [216, 256, 'SEVERITY'], [263, 285, 'COMPLIANT RISED']]}]


In [13]:
for batch in data:
    text = batch[0]  # Extract text
    entities = batch[1]["entities"]  # Extract entities

    print("Sentence:", text)
    print("Entities:", entities)


Sentence: I am a farmer from Tamil Nadu. My land was affected by floods, and my patta documents are missing. I have applied for a duplicate patta, but my application has been pending for 11 months. Without this document, I am unable to claim government compensation. I had filed complaints twice, but no action was taken.
Entities: [[67, 135, 'ISSUE'], [165, 187, 'DELAY'], [216, 256, 'SEVERITY'], [263, 285, 'COMPLIANT RISED']]
Sentence: I am a farmer from Tamil Nadu. My land was affected by floods, and my patta documents are missing. I have applied for a duplicate patta, but my application has been pending for 12 months. Without this document, I am unable to claim government compensation. I had filed complaints twice, but no action was taken.
Entities: [[67, 135, 'ISSUE'], [165, 187, 'DELAY'], [216, 256, 'SEVERITY'], [269, 285, 'COMPLIANT RISED']]
Sentence: I applied for a caste certificate for my child’s school admission 11 months ago. The delay in processing is causing problems in secu

In [14]:
from transformers import AutoTokenizer

# Load the fast tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased", use_fast=True)

# Define label mapping
label_map = {"O": 0, "ISSUE": 1, "DELAY": 2, "SEVERITY": 3, "COMPLIANT RISED": 4}

tokenized_data = []
for batch in data:
    text = batch[0]  # Extract text
    entities = batch[1]["entities"]  # Extract entities

    # Tokenize the text
    tokenized = tokenizer(
        text,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
        return_offsets_mapping=True  # Now works with `use_fast=True`
    )

    # Initialize labels with "O" (outside entity) by default
    labels = ["O"] * len(tokenized["input_ids"][0])

    # Assign correct entity labels based on offset mapping
    for start, end, entity_type in entities:
        for i, (token_start, token_end) in enumerate(tokenized["offset_mapping"][0]):
            if token_start >= start and token_end <= end:
                labels[i] = entity_type  # Assign entity label

    # Convert labels to numerical format
    labels = [label_map[label] for label in labels]

    # Add tokenized data with labels
    tokenized_data.append({
        "input_ids": tokenized["input_ids"],
        "attention_mask": tokenized["attention_mask"],
        "offset_mapping": tokenized["offset_mapping"],
        "labels": labels
    })

print(tokenized_data[0])  # Check output


{'input_ids': tensor([[  101,  1045,  2572,  1037,  7500,  2013,  6008, 10703,  1012,  2026,
          2455,  2001,  5360,  2011, 14295,  1010,  1998,  2026,  6986,  2696,
          5491,  2024,  4394,  1012,  1045,  2031,  4162,  2005,  1037, 24473,
          6986,  2696,  1010,  2021,  2026,  4646,  2038,  2042, 14223,  2005,
          2340,  2706,  1012,  2302,  2023,  6254,  1010,  1045,  2572,  4039,
          2000,  4366,  2231,  9430,  1012,  1045,  2018,  6406, 10821,  3807,
          1010,  2021,  2053,  2895,  2001,  2579,  1012,   102,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,  

In [15]:
print(type(tokenizer))


<class 'transformers.models.bert.tokenization_bert_fast.BertTokenizerFast'>


In [16]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import AutoModelForTokenClassification

# Define custom dataset
class CustomNERDataset(Dataset):
    def __init__(self, tokenized_data):
        self.tokenized_data = tokenized_data

    def __len__(self):
        return len(self.tokenized_data)

    def __getitem__(self, idx):
        item = self.tokenized_data[idx]
        return {
            "input_ids": item["input_ids"].squeeze(0),  # Remove batch dimension
            "attention_mask": item["attention_mask"].squeeze(0),
            "labels": torch.tensor(item["labels"]),
        }

# Define collate function for batching
def collate_fn(batch):
    input_ids = torch.stack([item["input_ids"] for item in batch])
    attention_mask = torch.stack([item["attention_mask"] for item in batch])
    labels = torch.nn.utils.rnn.pad_sequence(
        [item["labels"] for item in batch], batch_first=True, padding_value=0
    )  # Pad labels with 0 (O)

    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}

# Create dataset and DataLoader
dataset = CustomNERDataset(tokenized_data)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)

# Load BERT model for token classification
model = AutoModelForTokenClassification.from_pretrained("bert-base-uncased", num_labels=len(label_map))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
loss_fn = torch.nn.CrossEntropyLoss()

# Training loop
epochs = 3
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in dataloader:
        input_ids, attention_mask, labels = (
            batch["input_ids"].to(device),
            batch["attention_mask"].to(device),
            batch["labels"].to(device),
        )

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(dataloader)}")

# Save trained model
model.save_pretrained("ner_model")
tokenizer.save_pretrained("ner_model")


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Loss: 0.6875050343573094
Epoch 2, Loss: 0.4091581702232361
Epoch 3, Loss: 0.34303464367985725


('ner_model/tokenizer_config.json',
 'ner_model/special_tokens_map.json',
 'ner_model/vocab.txt',
 'ner_model/added_tokens.json',
 'ner_model/tokenizer.json')

In [28]:
import torch
import re
import nltk
from nltk.corpus import stopwords
from transformers import AutoTokenizer

nltk.download("stopwords")
STOPWORDS = set(stopwords.words("english"))

device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "dbmdz/bert-large-cased-finetuned-conll03-english"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

ISSUE_KEYWORDS = [
    "patta", "mutation", "records", "certificate", "heir", "land", "ownership", "name transfer",
    "survey", "demarcation", "boundary", "title", "correction", "change", "partition",
    "encumbrance", "fmb", "revenue", "sub-division", "conversion", "classification"
]

DELAY_KEYWORDS = [
    "pending", "delayed", "months", "years", "not processed", "waiting", "stuck", "approval"
]

SEVERITY_KEYWORDS = [
    "unable", "problems", "disputes", "correction", "compensation", "financial loss",
    "legal issue", "hindrance", "obstruction", "transaction failed", "claim rejected",
    "ownership conflict"]

COMPLAINT_KEYWORDS = [
    "submitted", "follow-ups", "filed", "raised"
]

def assign_label(token):
    """Assigns a label based on keyword matching."""
    token_lower = token.lower()

    if any(word in token_lower for word in ISSUE_KEYWORDS):
        return "ISSUE"
    elif any(word in token_lower for word in DELAY_KEYWORDS):
        return "DELAY"
    elif any(word in token_lower for word in SEVERITY_KEYWORDS):
        return "SEVERITY"
    elif any(word in token_lower for word in COMPLAINT_KEYWORDS):
        return "COMPLAINT"

    return "O"

def predict_entities(text):
    text_lower = text.lower()

    tokenized = tokenizer(text, return_offsets_mapping=True)
    tokens = tokenized["input_ids"]
    offset_mapping = tokenized["offset_mapping"]

    words = tokenizer.convert_ids_to_tokens(tokens)

    merged_words = []
    current_word = ""
    for word in words:
        if word.startswith("##"):
            current_word += word[2:]
        else:
            if current_word:
                merged_words.append(current_word)
            current_word = word
    if current_word:
        merged_words.append(current_word)

    entities = []
    for word in merged_words:
        word_lower = word.lower()
        if word_lower in STOPWORDS or word in {".", ",", ";", ":"}:
            continue

        if word_lower in ISSUE_KEYWORDS:
            entities.append((word, "ISSUE"))
        elif word_lower in DELAY_KEYWORDS:
            entities.append((word, "DELAY"))
        elif word_lower in SEVERITY_KEYWORDS:
            entities.append((word, "SEVERITY"))
        elif word_lower in COMPLAINT_KEYWORDS:
            entities.append((word, "COMPLAINT"))

    return entities

sentence = "My patta document is pending for months. I filed a complaint but got no correction, causing financial transactions issues."
predicted_entities = predict_entities(sentence)
print("Predicted Entities:", predicted_entities)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Predicted Entities: [('patta', 'ISSUE'), ('pending', 'DELAY'), ('months', 'DELAY'), ('filed', 'COMPLAINT'), ('correction', 'ISSUE')]
